<a href="https://colab.research.google.com/github/Mullen44/Segmentation-Ventricle/blob/main/ventSegUnetMain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# GPU Allocation


In [ ]:
# memory footprint support libraries/code
!ln -sf /opt/bin/nvidia-smi /usr/bin/nvidia-smi
!pip install gputil
!pip install psutil
!pip install humanize
import psutil
import humanize
import os
import GPUtil as GPU
# retrieve GPU type
GPUs = GPU.getGPUs()
gpu = GPUs[0]
# check allocation of GPU resources
def printm():
 process = psutil.Process(os.getpid())
 print("Gen RAM Free: " + humanize.naturalsize( psutil.virtual_memory().available ), " | Proc size: " + humanize.naturalsize( process.memory_info().rss))
 print("GPU RAM Free: {0:.0f}MB | Used: {1:.0f}MB | Util {2:3.0f}% | Total {3:.0f}MB".format(gpu.memoryFree, gpu.memoryUsed, gpu.memoryUtil*100, gpu.memoryTotal))
printm()

# Mount Drive / Load Modules


In [ ]:
# Mount google drive
from google.colab import drive
drive.mount('/content/gdrive', force_remount = True)

In [ ]:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

import os
import pandas as pd
import numpy as np
from scipy.io import loadmat
from scipy.spatial.distance import dice
import random
from PIL import Image
from matplotlib.pyplot import imsave, imread
import matplotlib.pyplot as plt
import cv2
import csv

%tensorflow_version 1.x
import tensorflow
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, TensorBoard, CSVLogger, LearningRateScheduler
from tensorflow.keras.optimizers import Adam
from tensorflow.python.keras.utils.data_utils import Sequence
%load_ext tensorboard

print(tensorflow.__version__)

%cd /content/gdrive/Shared drives/Andrew Mullen/Ventricle Segmentation/
from unet import *
from unet3 import *
from utilities import *
from data_generator_keras import *

In [ ]:
# Authenticate and create the PyDrive client
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [ ]:
%cd /content/gdrive/My Drive/NeuralNetworkTools/
!pip install SimpleITK
from data_generator import *
from loss_functions import *
from models import *
from utils import *

# Divide Data


In [ ]:
%cd /content/gdrive/Shared drives/Andrew Mullen/Ventricle Segmentation/
# Specify the path of the ground truths and volumes
VOL_path = 'Data/Standardized/v2/'
GT_path = 'Data/Ground Truths/'
BRAIN_path = 'Data/Masks/Brain/U-Net/'

VOL, GT = createFileList(vol_path=VOL_path, gt_path=GT_path, brain_path=BRAIN_path)

In [ ]:
# Initialize
TrainPercent = 0.85
ValPercent = 0.2

# Divide into train and test datasets
DATA, Train_Index, Val_Index = divideData(train_percentage=TrainPercent, val_percentage=ValPercent, vol=VOL, gt=GT)

In [ ]:
# Divide train_vol/GT into layers/images
Train_Path = 'Data/train/'
Val_Path = 'Data/val/'
Test_Path = 'Data/test/'

Train_Len = 0
Val_Len = 0


Vol_Dir = os.listdir(VOL_path)

# Check if directory is empty before refilling it
if len(os.listdir(Train_Path + 'images')) == 0 :
  print('Saving...')
  # Save images to Directory
  for i in range(len(VOL)) :
    if i not in Train_Index and i not in Val_Index :
      # Save to test folder
      Path_Image = Test_Path + 'inputs/' + Vol_Dir[i][:-4]
      Path_Label = Test_Path + 'targets/' + Vol_Dir[i][:-4]

      image = VOL[i].astype(np.float32)
      label = GT[i].astype(np.uint8)

      image = (image-np.amin(image))/(np.amax(image)-np.amin(image))

      np.save(Path_Image, image)
      np.save(Path_Label, label)
    else :
      for j in range(VOL[i].shape[2]) :
        if i in Train_Index :
          # Save to train folder
          Path_Image = Train_Path + 'images/' + Vol_Dir[i][:-4] + '_slice_' + str(j) +'.png'
          Path_Label = Train_Path + 'labels/' + Vol_Dir[i][:-4] + '_slice_' + str(j) +'.png'

          #image = Image.fromarray(VOL[i][:,:,j].astype(np.uint8))
          #image.convert('gray')
          #label = Image.fromarray(VOL[i][:,:,j].astype(np.uint8))

          #image.save(Path_Image)
          #label.save(Path_Label)

          image = VOL[i][:,:,j].astype(np.float32)
          label = GT[i][:,:,j].astype(np.uint8)

          image = (image-np.amin(image))/(np.amax(image)-np.amin(image))

          print(np.unique(label))

          imsave(Path_Image, image, cmap='gray', origin='upper')
          imsave(Path_Label, label, cmap='gray', origin='upper')

        elif i in Val_Index :
          # Save to val folder
        
          Path_Image = Val_Path + 'images/' + Vol_Dir[i][:-4] + '_slice_' + str(j) +'.png'
          Path_Label = Val_Path + 'labels/' + Vol_Dir[i][:-4] + '_slice_' + str(j) +'.png'

          #image = Image.fromarray(VOL[i][:,:,j].astype(np.uint8))
          #label = Image.fromarray(VOL[i][:,:,j].astype(np.uint8))

          #image.save(Path_Image)
          #label.save(Path_Label)

          image = VOL[i][:,:,j].astype(np.float32)
          label = GT[i][:,:,j].astype(np.uint8)

          image = (image-np.amin(image))/(np.amax(image)-np.amin(image))

          print(np.unique(label))

          imsave(Path_Image, image, cmap='gray', origin='upper') #vmin=0.0, vmax=255.0,
          imsave(Path_Label, label, cmap='gray', origin='upper') #vmin=0.0, vmax=255.0,


else : 
  print('Already Filled')


#Model Training


In [ ]:
#model = Unet()
choice = 'uresnet'
model, params = select_model(choice = choice)
model.summary()

In [ ]:
def scheduler(epoch) :
  if epoch < 40 :
    return 5e-5
  if epoch >= 40 :
    return 1e-5


In [ ]:
# Initialize
FILEPATH = '/content/gdrive/Shared drives/Andrew Mullen/Ventricle Segmentation/Models/'
EPOCHS = 100
STEPS = 70 #len(DATA['train_vol']) // BS
valSTEPS = 15 #len(DATA['val_vol']) // BS
BS = 32 # 1 = Optimal
lr = 1e-4
Model_Name = choice + '_batch_' + str(BS) + '_lr_' + str(lr) + '/'
FILEPATH = FILEPATH + Model_Name

# Specify dictionary of arguments for ImageDataGenerator Class
train_args = dict(rotation_range=0.2,
                  width_shift_range=0.05,
                  height_shift_range=0.05,
                  shear_range=0.05,
                  zoom_range=0.05,
                  fill_mode='nearest',
									rescale = 1.0/255) # Try changing this to 1./255

val_args = dict(rescale = 1.0/255)

# Create the generators
train_gen = train_generator(Train_Path, train_args, batch_size=BS)
val_gen = val_generator(Val_Path, val_args, batch_size=BS)


if os.path.isdir(FILEPATH) == 0 :
  os.mkdir(FILEPATH)
  print('Creating Directory')

# Set Callbacks
model_checkpoint = ModelCheckpoint(FILEPATH + 'model.hdf5', monitor='val_loss', save_best_only=True)
early_stop = EarlyStopping(monitor = 'val_loss', patience = 8)
t_board = TensorBoard(FILEPATH + './logs')
csv_logger = CSVLogger(FILEPATH + 'model_test.csv')
schedule = LearningRateScheduler(scheduler, verbose=1)

CALLBACKS = [model_checkpoint, early_stop, t_board, csv_logger]

# Compile Model
model.compile(optimizer=Adam(lr), loss=dice_loss, metrics=[DSC]) 

# Fit Model
model_history = model.fit_generator(train_gen, 
                    validation_data=val_gen, 
                    validation_steps=valSTEPS, 
                    steps_per_epoch=STEPS, 
                    epochs=EPOCHS, 
                    callbacks=[model_checkpoint, early_stop, t_board, csv_logger])#, schedule])
                                   

# Plot Loss Curves

In [ ]:
FILEPATH = '/content/gdrive/Shared drives/Andrew Mullen/Ventricle Segmentation/Models/uresnet_batch_4_lr_0.0001/'
CSVfile = FILEPATH + 'model_test.csv'
#CSVfile = '/content/gdrive/Shared drives/Andrew Mullen/Ventricle Segmentation/Models/uresnet_batch_16_epochs_100_steps_139_lr_0.0001model_test.csv'
model_curves = pd.read_csv(CSVfile)

In [ ]:
print(min(model_curves['loss']))
print(max(model_curves['DSC']))
print(min(model_curves['val_loss']))
print(max(model_curves['val_DSC']))

In [ ]:
# Plot loss vs epochs
plt.plot(model_curves['loss'], label='Training Loss')
plt.plot(model_curves['val_loss'], label='Validation Loss')
plt.ylabel('Dice Loss')
plt.xlabel('Epochs')
plt.title('Loss Curve')
plt.legend()
plt.show()

# Plot DSC vs epochs
plt.plot(model_curves['DSC'], label='Training DSC')
plt.plot(model_curves['val_DSC'], label='Validation DSC')
plt.ylabel('DSC')
plt.xlabel('Epochs')
plt.title('DSC Curve')
plt.legend()
plt.show()

In [ ]:
FILEPATH = '/content/gdrive/Shared drives/Andrew Mullen/Ventricle Segmentation/Models/Unet3_batch_16_epochs_100_steps_139_sched/'
CSVfile = FILEPATH + 'model_test.csv'
BS_16 = pd.read_csv(CSVfile)

FILEPATH = '/content/gdrive/Shared drives/Andrew Mullen/Ventricle Segmentation/Models/Unet3_batch_8_epochs_100_steps_280_sched/'
CSVfile = FILEPATH + 'model_test.csv'
BS_8 = pd.read_csv(CSVfile)

FILEPATH = '/content/gdrive/Shared drives/Andrew Mullen/Ventricle Segmentation/Models/Unet3_batch_4_epochs_100_steps_560_sched/'
CSVfile = FILEPATH + 'model_test.csv'
BS_4 = pd.read_csv(CSVfile)

FILEPATH = '/content/gdrive/Shared drives/Andrew Mullen/Ventricle Segmentation/Models/Unet3_batch_2_epochs_100_steps_1120_sched/'
CSVfile = FILEPATH + 'model_test.csv'
BS_2 = pd.read_csv(CSVfile)

f, axarr = plt.subplots(2,2,figsize=(15,10))
axarr[0,0].plot(BS_16['val_DSC'], '-r', label='BS_16 - Val DSC')
axarr[0,0].plot(BS_8['val_DSC'], '-b', label='BS_8 - Val DSC')
axarr[0,0].plot(BS_4['val_DSC'], '-g', label='BS_4 - Val DSC')
axarr[0,0].plot(BS_2['val_DSC'], '-k', label='BS_2 - Val DSC')
axarr[0,0].legend()
axarr[0,0].set_title('Validation DSC')
axarr[0,0].set(ylabel='DSC',xlabel='Epochs')

#plt.figure(figsize=(12,7))
axarr[0,1].plot(BS_16['DSC'], '-r', label='BS_16')
axarr[0,1].plot(BS_8['DSC'], '-b', label='BS_8')
axarr[0,1].plot(BS_4['DSC'], '-g', label='BS_4')
axarr[0,1].plot(BS_2['DSC'], '-k', label='BS_2')
axarr[0,1].legend()
axarr[0,1].set_title('Train DSC')
axarr[0,1].set(ylabel='DSC',xlabel='Epochs')

#plt.figure(figsize=(12,7))
axarr[1,0].plot(BS_16['val_loss'], '-r', label='BS_16')
axarr[1,0].plot(BS_8['val_loss'], '-b', label='BS_8')
axarr[1,0].plot(BS_4['val_loss'], '-g', label='BS_4')
axarr[1,0].plot(BS_2['val_loss'], '-k', label='BS_2')
axarr[1,0].legend()
axarr[1,0].set_title('Validation Loss')
axarr[1,0].set(ylabel='Loss',xlabel='Epochs')

#plt.figure(figsize=(12,7))
axarr[1,1].plot(BS_16['loss'], '-r', label='BS_16')
axarr[1,1].plot(BS_8['loss'], '-b', label='BS_8')
axarr[1,1].plot(BS_4['loss'], '-g', label='BS_4')
axarr[1,1].plot(BS_2['loss'], '-k', label='BS_2')
axarr[1,1].legend()
axarr[1,1].set_title('Training Loss')
axarr[1,1].set(ylabel='DSC',xlabel='Epochs')

for ax in f.get_axes():
    ax.label_outer()

# Test Model


In [ ]:
# File Paths
vol_path = '/content/gdrive/Shared drives/Andrew Mullen/Ventricle Segmentation/Data/test/inputs/'
gt_path = '/content/gdrive/Shared drives/Andrew Mullen/Ventricle Segmentation/Data/test/targets/'
model_path = '/content/gdrive/Shared drives/Andrew Mullen/Ventricle Segmentation/Models/uresnet_batch_8_lr_0.0001/model.hdf5'
#model_path = '/content/gdrive/Shared drives/Andrew Mullen/Ventricle Segmentation/Models/uresnet_batch_16_epochs_100_steps_139_lr_0.0001model.hdf5'

# Load model
model, params = select_model(choice = 'uresnet')
#model = Unet3()
model.load_weights(model_path)

# Make Predictions
i = 0
DSC = []
EF = []
predict = []
ground = []
name = []
for file in os.listdir(vol_path) :
  print(file)
  name.append(file)

  # Load Data
  vol = np.load(vol_path + file)
  test_gt = np.load(gt_path + file)
  print('Vol shape:', vol.shape)

  # Normalize/Binarize GT
  test_gt = test_gt/np.max(test_gt)
  test_gt[test_gt>0.5] = 1
  test_gt[test_gt<=0.5] = 0
  test_gt = test_gt.astype(np.uint8)
  ground.append(test_gt)
  #print('Volume max and min:', np.max(test_vol), np.min(test_vol), test_vol.shape, 'GT max and min:', np.max(test_gt), np.min(test_gt), np.unique(test_gt), test_gt.shape)

  # Save originial dimensions
  vol_size = (vol.shape[0:2])

  # Make Predictions
  test_pred = getPred(vol, model)#, img_size=(vol_size[0], vol_size[1]))

  pred = np.zeros(test_gt.shape)

  for j in range(test_pred.shape[2]) :
    pred[:,:,j] = cv2.resize(test_pred[:,:,j], (test_gt.shape[1], test_gt.shape[0]), interpolation=cv2.INTER_LINEAR)
    pred[:,:,j] = Binarize(pred[:,:,j], 0.5)

  #test_pred = Binarize(test_pred, 0.5)
  predict.append(pred)

  print(test_gt.shape, pred.shape)

  # Evaluate predictions
  y_true = test_gt.flatten()
  y_pred = pred.flatten()
  print('y_true shape:', y_true.shape, 'y_pred shape:', y_pred.shape)
  DSC.append(1 - dice(y_true, y_pred))
  i+=1



In [ ]:
x_dim = [0.8594, 0.8594, 0.8594, 0.8594, 0.8594, 0.8594, 1, 0.4286, 0.8594, 0.4286, 1]
y_dim = x_dim
z_dim = [5, 5, 5, 5, 5, 3, 3, 5, 3, 3, 3]
volGT = []
volPred = []
VOL = {}


for i in range(len(ground)) :
  vol = (x_dim[i]*y_dim[i]*z_dim[i])*len(ground[i][ground[i]==1])
  volGT.append(vol/1000)
  vol = (x_dim[i]*y_dim[i]*z_dim[i])*len(predict[i][predict[i]==1])
  volPred.append(vol/1000)
  VOL[name[i]] = vol

In [ ]:
EF = {}
D = {}
for i in range(len(predict)) :
  TP, FN, TN, FP = performance_measure(ground[i], predict[i])
  EF[name[i]] = (FP / (TP + FN))
  D[name[i]] = DSC[i]


print(EF)
print(D)
print(VOL)


In [ ]:
# Save Dictionary Files

w = csv.writer(open("DSC.csv", "w"))
for key, val in D.items():
  w.writerow([key, val])

a = csv.writer(open("EF.csv", "w"))
for key, val in EF.items():
  a.writerow([key, val])

b = csv.writer(open("Volume.csv", "w"))
for key, val in VOL.items():
  b.writerow([key, val])

In [ ]:
for i in range(len(DSC)) :
  print(i, name[i], DSC[i], EF[i], volPred[i], volGT[i])

In [ ]:
print(sum(DSC)/len(DSC))

In [ ]:
x = EF.values()
sum(x)/11

# Statistical Evaluation

In [ ]:
vol=0
layer=28
img=DATA['test_vol']
gt=DATA['test_gt']
pred=predict

print(name[vol])

plotImgGtPred(vol=vol, layer=layer, img=img, gt=gt, pred=pred)

img_1 = DATA['test_gt'][vol][:,:,layer]
img_2 = predict[vol][:,:,layer]
plt.figure()
plt.imshow(img_1)
plt.imshow(img_2, alpha=0.5)
plt.title('Overlay')


In [ ]:
DSC_unsup = np.array(loadmat('/content/gdrive/Shared drives/Andrew Mullen/Ventricle Segmentation/Data/DSC_uns_test.mat')['DSC'])
a = pd.read_csv('/content/gdrive/Shared drives/Andrew Mullen/Ventricle Segmentation/Data/DSC_Unet.csv')
b = pd.read_csv('/content/gdrive/Shared drives/Andrew Mullen/Ventricle Segmentation/Data/DSC_Uresnet.csv')
print(b)

DSC_Unet = [0.9381, 0.9580, 0.9421, 0.9368, 0.9247, 0.9263, 0.8734, 0.8886, 0.8947, 0.8347, 0.8654]
DSC_Uresnet = [0.9435, 0.9475, 0.9151, 0.9181, 0.9026, 0.8843, 0.8287, 0.8667, 0.8752, 0.8215, 0.8241]

print(len(DSC_Uresnet))


data = [DSC_unsup, DSC_Unet, DSC_Uresnet]
plt.figure()
plt.boxplot(data)
plt.xticks([1, 2, 3],['Unsupervised','Unet', 'Uresnet'])
plt.ylim(0, 1)
plt.ylabel('Dice Score')
plt.title('DSC')
plt.imshow


In [ ]:
EF_unsup = np.array(loadmat('/content/gdrive/Shared drives/Andrew Mullen/Ventricle Segmentation/Data/EF_unsup.mat')['EF'])
EF_unsup = np.squeeze(EF_unsup)
a = pd.read_csv('/content/gdrive/Shared drives/Andrew Mullen/Ventricle Segmentation/Data/EF_Unet.csv')
b = pd.read_csv('/content/gdrive/Shared drives/Andrew Mullen/Ventricle Segmentation/Data/EF_Uresnet.csv')
print(EF_unsup)

EF_Unet = [0.0053,  0.0213, 0.0423, 0.0511, 0.0510, 0.0777, 0.1784, 0.0475, 0.0317, 0.0731, 0.0988]
EF_Uresnet = [0.0121, 0.0311, 0.0556, 0.0768, 0.0678, 0.1834, 0.2348, 0.0625, 0.0867, 0.1597, 0.1715]

print(len(EF_Uresnet))


data = [EF_unsup, EF_Unet, EF_Uresnet]
plt.figure()
plt.boxplot(data)
plt.xticks([1, 2, 3],['Unsupervised','Unet', 'Uresnet'])
plt.ylim(0, 1)
plt.ylabel('EF')
plt.title('Extra Fraction')
plt.imshow


In [ ]:
print(sum(DSC_Uresnet)/11, sum(EF_Uresnet)/11)

In [ ]:
DSC = np.array(DSC)
print(len(DSC))
DSC_philip = DSC[[1,3,7,9]]
DSC_siem = DSC[[2,4,6,10]]
DSC_ge = DSC[[0,5,8]]
print(DSC_philip, len(DSC_philip))
data = [DSC_ge, DSC_philip, DSC_siem]
plt.boxplot(data)
plt.xticks([1,2,3],['GE', 'Philips', 'Siemens'])
plt.title('DSC by MRI Manufacturer')
plt.ylabel('DSC')

In [ ]:
# Volume predicted + Volume ground truth vs ground truth

DSC = np.array(DSC)
volGT = np.array(volGT)
volPred = np.array(volPred)

plt.figure()
plt.scatter(volGT, DSC)
m, b = np.polyfit(volGT, DSC, 1)
plt.plot(volGT, m*volGT+b, color='red')
m = round(m, 5)
b = round(b, 5)
eq = 'y=' + str(m) + 'x+' + str(b)
plt.text(100, 0.92, eq, fontsize=12)
plt.title('DSC vs Ground Truth Volume')
plt.xlabel('Ground Truth Volume: cm^3')
plt.ylabel('DSC')
#plt.legend()

plt.figure()
plt.scatter(volPred, DSC)
m, b = np.polyfit(volPred, DSC, 1)
plt.plot(volPred, m*volPred+b, color='red')
m = round(m, 5)
b = round(b, 5)
eq = 'y=' + str(m) + 'x+' + str(b)
plt.text(82, 0.92, eq, fontsize=12)
plt.title('DSC vs Predicted Volume')
plt.xlabel('Predicted Volume: cm^3')
plt.ylabel('DSC')

plt.figure()
plt.scatter(volGT, volPred)
m, b = np.polyfit(volGT, volPred, 1)
plt.plot(volGT, m*volGT+b, color='red')
m = round(m, 5)
b = round(b, 5)
eq = 'y=' + str(m) + 'x+' + str(b)
plt.text(100, 80, eq, fontsize=12)
plt.title('Predicted Volume vs Ground Truth Volume')
plt.xlabel('Ground Truth Volume: cm^3')
plt.ylabel('Predicted Volume: cm^3')

In [ ]:
testData = pd.read_excel('/content/gdrive/Shared drives/Andrew Mullen/Ventricle Segmentation/Data/TestImagesData.xlsx')
print(testData['Volume Matlab'])

In [ ]:
# Volume Agrement

vol1 = np.array(testData['Volume Matlab'])
vol2 = np.array(volGT)
matVolAgree = volAgree(vol1=vol1, vol2=vol2)
vol1 = np.array(volPred)
UnetVolAgree = volAgree(vol1=vol1, vol2=vol2)

print(matVolAgree)
print(UnetVolAgree)

performance =[]
y_pos = [0,1,3,4,6,7,9,10,12,13,15,16,18,19,21,22,24,25,27,28,30,31,33,34]
objects = ('Unsupervised', 'Unet', 'Unsupervised', 'Unet', 'Unsupervised', 'Unet', 'Unsupervised', 'Unet', 'Unsupervised', 'Unet', 'Unsupervised', 'Unet', 'Unsupervised', 'Unet', 'Unsupervised', 'Unet', 'Unsupervised', 'Unet', 'Unsupervised', 'Unet', 'Unsupervised', 'Unet', 'Average Unsupervised', 'Average Unet')

for i in range(len(matVolAgree)) :
  performance.append(matVolAgree[i])
  performance.append(UnetVolAgree[i])


performance.append(sum(matVolAgree)/len(matVolAgree))
performance.append(sum(UnetVolAgree)/len(UnetVolAgree))
bar = plt.bar(y_pos, performance)

bar[1].set_color('r')
bar[3].set_color('r')
bar[5].set_color('r')
bar[7].set_color('r')
bar[9].set_color('r')
bar[11].set_color('r')
bar[13].set_color('r')
bar[15].set_color('r')
bar[17].set_color('r')
bar[19].set_color('r')
bar[21].set_color('r')
bar[23].set_color('r')
plt.xticks(y_pos, objects, rotation='vertical')
plt.title('Volume Agreement Between Segmentations and Ground Truths')
plt.ylabel('Volume Agreement')

In [ ]:
print(performance[-2])
print(performance[-1])

In [ ]:
# Volume comparisons

#objects = ('Matlab', 'Unet', 'GT', 'Matlab', 'Unet', 'GT', 'Matlab', 'Unet', 'GT', 'Matlab', 'Unet', 'GT', 'Matlab', 'Unet', 'GT', 'Matlab', 'Unet', 'GT', 'Matlab', 'Unet', 'GT', 'Matlab', 'Unet', 'GT', 'Matlab', 'Unet', 'GT', 'Matlab', 'Unet', 'GT', 'Matlab', 'Unet', 'GT', )
#performance = []
#y_pos = np.array(range(33))
#tick=np.empty(33)
for i in range(len(testData)) :
  performance = [testData['Volume Matlab'][i], testData['Volume Unet'][i], testData['Volume GT'][i]]
  y_pos = (3*i+i, 3*i+i+1, 3*i+i+2)
  #tick.append(y_pos)

  plt.bar(y_pos, performance, align='center', alpha=0.5)

  #plt.xticks(y_pos, objects, rotation='vertical')
  plt.ylabel('cm^3')
  plt.title('Volume')

average = [sum(testData['Volume Matlab'])/11, sum(testData['Volume Unet'])/11, sum(testData['Volume GT'])/11]
y_pos = ([44,45,46])
plt.bar(y_pos, average, align='center', alpha=0.5)
objects = ('Matlab', 'Unet', 'GT', 'Matlab', 'Unet', 'GT', 'Matlab', 'Unet', 'GT', 'Matlab', 'Unet', 'GT', 'Matlab', 'Unet', 'GT', 'Matlab', 'Unet', 'GT', 'Matlab', 'Unet', 'GT', 'Matlab', 'Unet', 'GT', 'Matlab', 'Unet', 'GT', 'Matlab', 'Unet', 'GT', 'Matlab', 'Unet', 'GT', 'Average Matlab', 'Average Unet', 'Average GT')
#performance = []
y_pos = np.array([0,1,2,4,5,6,8,9,10,12,13,14,16,17,18,20,21,22,24,25,26,28,29,30,32,33,34,36,37,38,40,41,42,44,45,46])

plt.xticks(y_pos, objects, rotation='vertical')


In [ ]:
len(performance)

# Troubleshooting



In [ ]:
VOL_path = 'Data/Standardized/v2/'
GT_path = 'Data/Ground Truths/'
BRAIN_path = 'Data/Masks/Brain/U-Net/'

vol_dir = os.listdir(VOL_path)
loadmat(VOL_path + vol_dir[2])['im']['hdr']#['PixelSpacing']

In [ ]:
len(DSC_unsup[0])

In [ ]:
image = VOL[44][:,:,28]
label = GT[44][:,:,28]

imsave('/content/gdrive/Shared drives/Andrew Mullen/Ventricle Segmentation/image.png', image, cmap='gray', origin='upper')
imsave('/content/gdrive/Shared drives/Andrew Mullen/Ventricle Segmentation/label.png', label, cmap='gray', origin='upper')

image = imread('/content/gdrive/Shared drives/Andrew Mullen/Ventricle Segmentation/image.png')
label = imread('/content/gdrive/Shared drives/Andrew Mullen/Ventricle Segmentation/label.png')

f, axarr = plt.subplots(1, 2)
axarr[0].imshow(image, cmap="gray")
axarr[1].imshow(label, cmap="gray")

In [ ]:
import random as rd
x = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13]
rd.shuffle(x)

In [ ]:
print(x)